# Прогнозирование и оптимизация складских остатков

Базовая кафедра: Кафедра логистических систем и технологий  
Обучающийся: Шмунь М.С., Б03-907а  
Научный руководитель: Старостин Е.А.

## Введение

Доля Интернет-торговли в розничной торговле ежегодно увеличиваются: по данным Росстата (табл. 1), в 2022 году 6% от общего объема приходилась на продажи через Интернет. При этом сохраняются все предпосылки для дальнейшего развития этого тренда: по мере распространения и улучшения качества сервиса растет как доверие населения к совершению покупок через Интернет, так и интерес предпринимателей к реализации своих товаров и услуг в сети. Для сравнения, в Китае, где проникновение цифровых сервисов в жизнь граждан и их взаимная интеграция находится на куда более развитом уровне, эта доля в 2020 году составляла 30%. (рис. 2). 
Увеличивающийся объем Интернет-торговли приводит к более высокому уровню конкуренции на разных уровнях экономики, как среди отдельных продавцов, так и среди площадок: покупателю практически мгновенно доступен широкий ассортимент товаров, а также открытая информация о ценах и отзывы об уровне сервиса и качестве товаров, что вынуждает поставщиков поддерживать конкурентные уровни цен и качества обслуживания. Кроме того, в условиях нарастающих объемов даже небольшие оптимизации в операционных процессах могут стать значимым конкурентным преимуществом и ключом к росту доли продавца или площадки на рынке.
В этой работе были изучены и реализованы подходы по оптимизации управления складскими остатками на основе прогнозирования, которые могут быть использованы как отдельными продавцами-участниками электронной коммерции, так и площадками (маркетплейсами) для постановки целей поставщикам. Основной моделью для построения прогноза является ARIMA (англ. Auto-Regressive Integrated Moving Average), а также ее модификации с сезонными и экзогенными факторами.

Модели были построены на основе данных о фактических продажах товаров в десяти различных товарных категориях в кластере распределительного центра в Свердловской области за 2021-2022 годы и 1 квартал 2023 года. 
Модели оценивались как по статистическим показателям (MAPE, RMSE) так и по бизнес-показателям, рассчитанным по результатам моделирования: упущенная выручка, затраты на хранение излишков, уровень сервиса. Таким образом, оценивается не только синтетическая «точность» прогноза, но и влияние от внедрения модели на успешность предприятия, что может быть полезно для оценки подхода на соответствие целям и задачам бизнеса и принятия решения о внедрении той или иной модели. 

## Обзор данных

Основной набор данных представляет из себя таблицу с понедельным срезом продаж (в штуках и в рублях) со следующими столбцами: неделя (по понедельнику), название категории, продажи в рублях, продажи в штуках.
Кроме того, для моделей, учитывающих экзогенные факторы, предоставлены периоды распродаж и даты аномалий на макроуровне (начало СВО, мобилизация). Для каждой категории также указаны норма (в процентах от стоимости) прибыли площадки за реализацию и затрат на хранение одной товарной единицы.

**TODO** Сниппеты таблиц

## Обзор классических подходов

### Экспертные модели

**TODO**

### Модель экономичного размера заказа
Распространенной моделью управления товарными остатками является EOQ (англ. Economic Order Quantity – экономичный размер заказа). В этой модели заданы следующие условия: 
	Спрос на товары известен и постоянен
	Размер заказа не ограничен
	Доставка товара на склад производится мгновенно после заказа

В постановке задачи EOQ введены следующие обозначения: 
	d – фиксированный спрос за период
	с – цена продажи единицы товара;
	К – затраты на размещение заказа;
	h — издержки за хранение единицы товара на складе;
	Q – искомая величина заказа

Решение задачи известно и детерминировано:
$$ Q^*=\sqrt{(\frac{2Kd}{h})} $$

При этом заказ производится при обнулении остатка.

Такое сильное ограничение как постоянный известный спрос сильно ограничивает применимость модели EOQ в реальных задачах, поэтому сейчас такой подход считается устаревшим, и чаще применяются модели, где спрос представлен либо детерминированной функции от внешних переменных (время, цена и т.д.), либо случайный


### Модели с фиксированными параметрами

К таким моделям управления остатками можно отнести модели с фиксированными промежутками между заказами (где поставщик управляет размером заказа) и с фиксированным размером заказа (где поставщик управляет моментом оформления заказа). Такие модели применяются в случае, если ограничения параметров обусловлены внешними факторами, например условиями работы производителя, возможностями логистической сети и т.д.


## Модели на основе прогнозирования спроса

Для моделей на основе прогнозирования спрос в периоде полагается случайной величиной с некоторой (не обязательно известной) функцией распределения, а сам ряд измерений спроса представляется экземпляром случайным процессом. Для такой задачи также характерен равный промежуток между заказами, например, минимально возможный, а также мгновенная доставка. В таких условиях очевидна стратегия поведения при идеальном прогнозировании: размещать заказ в точности равный спросу в ближайшем периоде. Таким образом полностью покрывается потребительский спрос, что максимизирует полученную выручку, и минимизируются затраты на хранение.  

В реальности же прогнозирование спроса на товары является сложной задачей: временные ряды такой природы редко являются стационарными, а также подвержены влиянию сезонных и внешних факторов. Для повышения качества прогнозирования применяются самые различные подходы: как в подготовке данных (логарифмирование, взятие разностей, очистка от выбросов и пропусков данных), так и в архитектуре прогнозных моделей (учет сезонности и тренда, авторегрессия, использования рекуррентных и сверточных нейронных сетей).  

При этом ошибка прогнозирования влияет на прибыль ассиметрично: при избыточном запасе предприятие теряет только стоимость хранения излишка, а при недостаточном – всю выручку от реализации непроданных единиц и связанную с этим прибыль. В условиях высокой конкуренции продавцов покупатель, обнаружив дефицит товара у поставщика, скорее всего просто купит его у конкурента (а конкурент, обнаружив дефицит – повысит цены и получит дополнительную прибыль от реализации).  

Таким образом, в процессе управления товарными запасами возникает необходимость не только прогнозировать спрос (результатом прогноза обычно является среднее или наиболее вероятное значение случайной величины), но и интерпретировать полученный прогноз для минимизации суммарных потерь или достижения других целевых показателей: например, использовать верхние границы доверительного интервала для обеспечения заданной нормы уровня сервиса и минимизации упущенной прибыли.


Простейшей моделью для прогнозирования спроса можно считать скользящее среднее (SMA – simple moving average, простое скользящее среднее) где прогнозом на период считается среднее значение нескольких предыдущих периодов

$$ SMA_i= \frac{1}{N} \sum_{k=1}^{N} y_{(i-k)} $$



Размер окна n при этом можно выбрать экспертно, либо оптимизировать на обучающей выборке. Следует также упомянуть модификации SMA – EMA и WMA (exponential moving average, weighted moving average), где предыдущие периоды суммируются с некоторыми весами. Аналогично, и размер окна, и веса можно как задать экспертно, так и найти оптимальные с помощью соответствующих методов. 

Более сложной моделью прогнозирования является метод экспоненциального сглаживания, в которой прогноз рассчитывается на основе не только фактических значений прошлых периодов, но и прогнозных. <code> [Модель](https://otexts.com/fpp2/ses.html) </code>. Модель имеет вид

$$ \^y_{t+h|t} = l_t = \alpha y_t + (1-\alpha)l_{t-1}  , \, где $$



$ y_t - фактические \, значения $  
$ \^y_{t+h|t} = l_t - прогнозные \, значения $  

а параметры $ \alpha \in (0, 1) , l_0 $ заданы изначально и могут быть оптимизированы.

Широко используемой моделью, которая основана на принципах экспоненциального сглаживания  модель Хольта-Винтерса, в которой ряд разделяется на сглаженную L, трендовую B и сезонную S составляющие с соответствующими коэффициентами $ \alpha, \beta, \gamma $. В общем случае модель используется для прогнозирования на несколько периодов с помощью временного параметра h. <code> [Модель](https://otexts.com/fpp2/holt-winters.html) </code>

Модель с аддитивным типом сезонности имеет вид:

$$ \^y_{t+h|t} = l_t + hb_t + s_{t+h-m(k+1)},  \,  где $$
    
$$ l_t = \alpha(y_t - s_{t-m}) + (1-\alpha)(l_{t-1}+b{t-1}) - экспоненциальное \, сглаживание $$  
$$ b_t = \beta^*(l_t - l_{t-1}) + (1-\beta^*)b{t-1} - трендовая \, составляющая $$  
$$ s_t = \gamma(y_t - l_{t-1} + (1-\gamma)s_{t-m}) - сезонная \, составляющая $$  
$$ k = mod (h-1)/m - сезонный  \, индекс $$  

Модель с мультипликативным видом сезонности имеет вид:

$$ \:y_{t+h|t} = (l_t + hb_t)s_{t+h-m(k+1)},  \,  где $$
    
$$ l_t = \alpha \frac{y_t}{s_{t-m}} + (1-\alpha)(l_{t-1}+b{t-1}) - экспоненциальное \, сглаживание $$
$$ b_t = \beta^*(l_t - l_{t-1}) + (1-\beta^*)b{t-1} - трендовая \, составляющая $$  
$$ s_t = \gamma\frac{y_t} {l_{t-1}+b_{t-1}} + (1-\gamma)s_{t-m}) - сезонная \, составляющая $$  
$$ k = mod (h-1)/m - сезонный  \, индекс $$  

Параметры $ \alpha, \beta, \gamma \in (0,1) $ также можно оптимизировать или зафиксировать извне. 

Модели из этого раздела (SMA, EMA, модель Хольта-Винтерса) будут использованы в качестве бенчмарка для сравнения с семейством моделей, описанных в следующем разделе.

## ARMA модель и ее модификации
### ARMA и ARIMA

Модель ARMA(p, q) (англ. Autoregressive moving-average model - Модель авторегрессии - скользящего среднего) можно представить как:

$$ y_{t+1} = ARMA(p,q) = C + AR_t(p) + MA_t(Q), \, где $$
$$ AR_t(p) = \sum_{k=1}^{p}y_{t-k}\phi_k - авторегрессионный \, член \,  порядка \,  p $$ 
$$ MA_t(q) = \sum_{k=1}^{q}\varepsilon_{t-k}\theta_k - случайно \, распределенная \,  ошибка \, $$
$$ с \, нулевым \, средним \, порядка \, q $$

В чистом виде модель может применяться только для стационарных процессов, т.е. не меняющих свойств случайной величины со временем. Как уже упоминалось, на практике стационарные ряды встречаются редко, в связи с чем обычно ARMA модель применяется не в чистом виде. Распространенным решением является взятие разности некоторого порядка от исходного ряда и моделирование, таким образом, не целевого значения, а его изменения во времени.  
Построенную таким образом модель обозначают как ARIMA(p,d,q), где p и q являются порядками AR и MA компонент, а d - порядок взятия разностей.   

Взятие разности можно порядка n рекуррентно обозначить как:
$$ \Delta^n y_t =  y^{(n)}_t = y^{(n-1)}_t - y^{(n-1)}_{t-1} $$

Таким образом, ARMA(p, q) = ARIMA(p, 0, q)

Порядки p, q можно выбирать на основе анализа автокорелляционной (ACF) и частичной автокорелляционной (PACF) функций. (Максимальные индексы значений ACF и PACF, отличных от нуля, являются рекомендованными порядками q и p соответственно).

### Проверка стационарности

При работе с временным рядом может оказаться, что ряд стационарен в своем исходном виде. С другой стороны,получив ряд разностей, мы все равно можем не добиться стационарности. В связи с этим возникает необходимость в проверке стационарности.

В данной работе для проверки будут использованы критерии ADF (Дики-Фуллера) и KPSS (Квятковского-Филлипса-Шмидта-Шина). Оба критерия тестируют гипотизы о (не-)стационарности временного ряда. 

**TODO**

### Модификации ARIMA

Модель ARIMA с учетом сезонности обозначается как $ SARIMA(p,d,q)\times(P,D,Q)_s $ , где P, D, Q - порядки авторегрессии, разности и скользящего среднего для сезонной компоненты, а s - показатель (период) сезонности.  <code> [Модель](https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html) </code>

$$ \phi_p(L)\~\phi_P(L^s)\Delta^d\Delta^D_sy_t = A(t) + \theta_q(L)\~\theta_Q(L^s)\epsilon_t $$  
$ \phi_p(L) - несезонная\,  AR\,  компонента\,  порядка\,  p$  
$ \~\phi_P(L^s) - сезонная\,  (т.е.\,  смещнная\,  на\,  s\,  лагов)\,  порядка\,  P$  
$ \Delta^d\Delta^D_sy_t = y^*_t - разность\, от\, исходного\, ряда\, порядка\, d\, и\, от\, смещенного\, на\, s\, порядка\, D $  
$ A(t) - тренд\,  (порядка\,  не\,  более\,  d) $  
$ \theta_q(L) - несезонный\,  компонент\,  скользящего\,  среднего\,  порядка\,  q$  
$ \theta_Q(L^s) - сезонный \, компонент\,  скользящего\,  среднего\,  порядка\,  Q$  


Аналогично исходной модели, порядок сезонной разности определяется на основании анализа стационарности ряда смещенных разностей, а порядки P,D - на основании периодически возникающих пиков PACF и ACF от исходного ряда. Глубину сезонности можно определить из субъективных соображений (напр., внутринедельная, квартальная, годовая сезонность и т.д.)

<code> [Модель](https://robjhyndman.com/hyndsight/arimax/) </code> ARMA с учетом внешних (экзогенных) факторов обозначается как ARMAX(p,q)

$$ y_t = c + \beta x_t + AR(p) + MA(q) $$
$ где\, x_t - ряд\, внешних\, факторов\, $


Наиболее общей <code> [моделью](https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html#ARIMA-Example-4:-ARMAX-(Friedman)) </code> , включающей в себя обе модификации, является SARIMAX:


$$ y_t = \beta x_t + u_t $$
$$ \phi_p(L)\~\phi_P(L^s)\Delta^d\Delta^D_su_t = A(t) + \theta_q(L)\~\theta_Q(L^s)\epsilon_t $$ 

